In [ ]:
# Colab-ready ReACT example: Ask the model for a short step-by-step plan, then code, then execute.

# 1) Install & import (run once per session)
!pip install openai --quiet

from openai import OpenAI
import re, io, contextlib
import os

# 2) Initialize client (replace with your key OR set environment variable OPENAI_API_KEY)
# Option A: set an env var in Colab first and uncomment the next line:
# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
# Option B: put the key directly (not recommended for shared notebooks)
client = OpenAI(api_key="YOUR_API_KEY_HERE")  # <-- replace this

# 3) The ReACT-style prompt (asks for a concise step-by-step plan, then only the code)
react_prompt = """
You are an expert Python developer. Provide a concise, numbered plan 3-5 short lines) describing how to
compute the nth Fibonacci number using an efficient iterative approach. After the plan, output ONLY the Python
code in a fenced ```python``` block. The code must define a function `compute(n)` that returns the nth Fibonacci
number and must include an `if __name__ == "__main__":` block that calls `compute(10)` and prints the result.
Do NOT include chain-of-thought or internal monologue — only a brief plan and the code block.
"""

# 4) Call the model
resp = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role":"user", "content": react_prompt}],
    temperature=0.0
)

text = resp.choices[0].message.content
print("=== RAW MODEL RESPONSE ===\n")
print(text)
print("\n=== END RAW RESPONSE ===\n")

# 5) Extract the plan (text before the first code fence) and the code (inside the first code fence)
plan = ""
code = None

if "```" in text:
    # split into before/after first fence
    before, after = text.split("```", 1)
    plan = before.strip()
    # the 'after' usually starts with "python\n" or "python\n<code>" or "<code>\n```..."
    # grab content up to the closing fence
    code_match = re.search(r"^(?:python\s*\n)?(.*)$", after, flags=re.S)
    if code_match:
        # extract everything up to the next ``` (if present)
        rest = after
        code_content = rest.split("```", 1)[0]
        code = code_content.strip()
else:
    # fallback: try to find "Code:" and take everything after that
    idx = text.lower().find("code:")
    if idx != -1:
        plan = text[:idx].strip()
        code = text[idx+5:].strip()

# 6) Show plan + code (if found)
print("=== PLAN (brief) ===")
print(plan or "(no plan found)")
print("\n=== GENERATED CODE ===")
if code:
    print(code)
else:
    print("No fenced code block found in the model response. Inspect raw response above.")

# 7) SAFELY execute the generated code and capture stdout output (user environment executes this)
if code:
    print("\n=== EXECUTING GENERATED CODE ===")
    exec_globals = {}
    f = io.StringIO()
    try:
        with contextlib.redirect_stdout(f):
            # Execute the generated code (it should define compute and run it in __main__ block)
            exec(code, exec_globals)
    except Exception as e:
        print("Execution error:", e)
    finally:
        output = f.getvalue()
        print(output if output else "(no output captured)")

print("\nDone. If you want the model to implement a different problem (factorial, primes, matrix ops), tell me which one and I’ll update the prompt/code.")
